# Chapter 08-02: TensorBoard 시각화

## 학습 목표
- TensorBoard를 설정하고 실행하는 방법을 익힌다
- 콜백을 활용한 자동 학습 로깅을 구현한다
- `tf.summary` API로 스칼라, 이미지, 히스토그램을 직접 기록한다
- 커스텀 학습 루프에서 TensorBoard를 활용한다

## 목차
1. TensorBoard 실행 방법
2. TensorBoard 콜백 자동 로깅
3. `tf.summary.scalar` 직접 사용
4. `tf.summary.image` - 이미지 로깅
5. `tf.summary.histogram` - 가중치 분포
6. 정리

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import datetime
import os

print(f"TensorFlow 버전: {tf.__version__}")

# 로그 디렉토리 기본 경로 설정
LOG_BASE_DIR = '/tmp/tb_logs'
os.makedirs(LOG_BASE_DIR, exist_ok=True)

# MNIST 데이터셋 로드 (실습 공통 사용)
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test  = X_test.astype('float32') / 255.0
# 채널 차원 추가
X_train = X_train[..., np.newaxis]
X_test  = X_test[..., np.newaxis]
print(f"학습 데이터: {X_train.shape}, 레이블: {y_train.shape}")

## 1. TensorBoard 실행 방법

### Jupyter Notebook에서 인라인 실행

```python
# Jupyter 매직 명령어로 TensorBoard 로드 및 실행
%load_ext tensorboard
%tensorboard --logdir logs
```

### 터미널에서 별도 실행

```bash
# 터미널에서 TensorBoard 서버 시작 (기본 포트 6006)
tensorboard --logdir ./logs

# 포트 지정
tensorboard --logdir ./logs --port 6007
```

브라우저에서 `http://localhost:6006` 접속하면 대시보드를 확인할 수 있다.

### 로그 디렉토리 구조 권장 패턴

```
logs/
├── run1/           ← 첫 번째 실험
│   ├── train/
│   └── validation/
└── run2/           ← 두 번째 실험 (하이퍼파라미터 변경)
    ├── train/
    └── validation/
```

## 2. TensorBoard 콜백으로 자동 로깅

`tf.keras.callbacks.TensorBoard`는 학습 루프에 자동으로 통합되어
손실, 정확도, 그래프, 가중치 히스토그램 등을 기록한다.

In [ ]:
# 간단한 CNN 모델 생성
def build_cnn():
    """MNIST 분류용 간단한 CNN 모델"""
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

model = build_cnn()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 타임스탬프로 고유한 로그 디렉토리 생성
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join(LOG_BASE_DIR, f'callback_{timestamp}')

# TensorBoard 콜백 설정
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,      # 매 에포크마다 가중치 히스토그램 기록
    write_graph=True,      # 모델 그래프 기록
    write_images=True,     # 가중치를 이미지로 시각화
    update_freq='epoch',   # 에포크 단위로 로그 업데이트
    profile_batch=0,       # 프로파일링 비활성화 (성능 오버헤드 방지)
)

# 학습 실행 (소량 데이터로 빠른 실습)
history = model.fit(
    X_train[:5000], y_train[:5000],
    validation_data=(X_test[:1000], y_test[:1000]),
    epochs=3,
    batch_size=64,
    callbacks=[tb_callback],
    verbose=1
)
print(f"\nTensorBoard 로그 위치: {log_dir}")
print("터미널에서 실행: tensorboard --logdir", LOG_BASE_DIR)

## 3. `tf.summary.scalar` - 커스텀 학습 루프에서 직접 기록

`model.fit`을 사용하지 않는 커스텀 학습 루프에서 메트릭을 직접 기록한다.

In [ ]:
# 커스텀 학습 루프용 모델 및 옵티마이저 초기화
custom_model = build_cnn()
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# summary writer 생성 (train / validation 분리)
custom_log_dir = os.path.join(LOG_BASE_DIR, f'custom_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
train_writer = tf.summary.create_file_writer(os.path.join(custom_log_dir, 'train'))
val_writer   = tf.summary.create_file_writer(os.path.join(custom_log_dir, 'validation'))

# tf.data 데이터셋 구성
train_dataset = tf.data.Dataset.from_tensor_slices((X_train[:2000], y_train[:2000]))
train_dataset = train_dataset.shuffle(1000).batch(64)

# 커스텀 학습 루프 (2 에포크)
for epoch in range(2):
    epoch_loss = 0.0
    num_batches = 0

    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            logits = custom_model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, custom_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, custom_model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)
        epoch_loss += loss.numpy()
        num_batches += 1

    avg_loss = epoch_loss / num_batches
    train_acc = train_acc_metric.result().numpy()
    train_acc_metric.reset_state()

    # tf.summary.scalar로 학습 메트릭 기록
    with train_writer.as_default():
        tf.summary.scalar('loss', avg_loss, step=epoch)
        tf.summary.scalar('accuracy', train_acc, step=epoch)

    # 검증 메트릭 계산 및 기록
    val_logits = custom_model(X_test[:500], training=False)
    val_loss = loss_fn(y_test[:500], val_logits).numpy()
    val_acc = tf.keras.metrics.sparse_categorical_accuracy(y_test[:500], val_logits)
    val_acc = tf.reduce_mean(val_acc).numpy()

    with val_writer.as_default():
        tf.summary.scalar('loss', val_loss, step=epoch)
        tf.summary.scalar('accuracy', val_acc, step=epoch)

    print(f"Epoch {epoch+1}: loss={avg_loss:.4f}, acc={train_acc:.4f} | val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

print(f"\n커스텀 루프 로그 위치: {custom_log_dir}")

## 4. `tf.summary.image` - 예측 결과 이미지 로깅

예측 결과나 중간 특징 맵을 이미지로 TensorBoard에 기록하면
모델이 무엇을 학습하고 있는지 직관적으로 확인할 수 있다.

In [ ]:
import matplotlib
matplotlib.use('Agg')  # 비대화형 백엔드 (서버 환경)
import matplotlib.pyplot as plt
import io

# 이미지 로거용 summary writer 생성
img_log_dir = os.path.join(LOG_BASE_DIR, f'images_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
img_writer = tf.summary.create_file_writer(img_log_dir)

def plot_predictions_to_image(images, true_labels, pred_labels, n=6):
    """예측 결과를 matplotlib 그림으로 그린 뒤 PNG 바이트로 반환"""
    fig, axes = plt.subplots(1, n, figsize=(12, 2))
    for i in range(n):
        axes[i].imshow(images[i, :, :, 0], cmap='gray')
        color = 'green' if true_labels[i] == pred_labels[i] else 'red'
        axes[i].set_title(f"실제:{true_labels[i]}\n예측:{pred_labels[i]}", color=color, fontsize=8)
        axes[i].axis('off')
    plt.tight_layout()

    # 그림을 PNG 바이트로 변환
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    plt.close(fig)
    return buf

# 샘플 이미지로 예측 수행
sample_images = X_test[:6]
sample_labels = y_test[:6]
preds = model.predict(sample_images, verbose=0)
pred_labels = np.argmax(preds, axis=1)

# matplotlib 그림 → TensorBoard 이미지 텐서 변환
buf = plot_predictions_to_image(sample_images, sample_labels, pred_labels)
image_tensor = tf.image.decode_png(buf.getvalue(), channels=4)
image_tensor = tf.expand_dims(image_tensor, 0)  # 배치 차원 추가

# TensorBoard에 이미지 기록
with img_writer.as_default():
    tf.summary.image('예측_결과_샘플', image_tensor, step=0)

# 원시 입력 이미지를 직접 기록 (배치 형태여야 함)
with img_writer.as_default():
    tf.summary.image('입력_이미지', sample_images[:4], step=0, max_outputs=4)

print(f"이미지 로그 저장 완료: {img_log_dir}")

## 5. `tf.summary.histogram` - 가중치 분포 로깅

가중치와 그래디언트의 분포를 에포크별로 기록하면
기울기 소실(Vanishing Gradient)이나 폭발(Exploding Gradient) 문제를 조기에 발견할 수 있다.

In [ ]:
# 히스토그램 로거용 summary writer
hist_log_dir = os.path.join(LOG_BASE_DIR, f'histogram_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
hist_writer = tf.summary.create_file_writer(hist_log_dir)

# 간단한 Dense 모델로 히스토그램 시연
hist_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(784,), name='dense_1'),
    tf.keras.layers.Dense(32, activation='relu', name='dense_2'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])
hist_optimizer = tf.keras.optimizers.Adam()
hist_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# 데이터 평탄화 (Dense 모델용)
X_flat_train = X_train[:2000].reshape(-1, 784)
X_flat_test  = X_test[:500].reshape(-1, 784)

# 히스토그램을 기록하는 커스텀 학습 루프
for epoch in range(3):
    dataset = tf.data.Dataset.from_tensor_slices((X_flat_train, y_train[:2000]))
    dataset = dataset.batch(128)

    for x_batch, y_batch in dataset:
        with tf.GradientTape() as tape:
            logits = hist_model(x_batch, training=True)
            loss = hist_loss_fn(y_batch, logits)
        grads = tape.gradient(loss, hist_model.trainable_variables)
        hist_optimizer.apply_gradients(zip(grads, hist_model.trainable_variables))

    # 에포크마다 가중치 및 그래디언트 히스토그램 기록
    with hist_writer.as_default():
        for var, grad in zip(hist_model.trainable_variables, grads):
            # 가중치 분포
            tf.summary.histogram(f'weights/{var.name}', var, step=epoch)
            # 그래디언트 분포 (None이 아닌 경우에만)
            if grad is not None:
                tf.summary.histogram(f'gradients/{var.name}', grad, step=epoch)
        # 에포크 손실도 함께 기록
        tf.summary.scalar('epoch_loss', loss, step=epoch)

    print(f"Epoch {epoch+1}/3 완료 | 마지막 배치 손실: {loss.numpy():.4f}")

print(f"\n히스토그램 로그 저장 완료: {hist_log_dir}")

## 6. 정리

### TensorBoard 주요 기능 요약

| 기능 | API | 확인 탭 |
|------|-----|----------|
| 손실/정확도 곡선 | 콜백 or `tf.summary.scalar` | Scalars |
| 모델 구조 그래프 | 콜백 `write_graph=True` | Graphs |
| 가중치 분포 | 콜백 `histogram_freq=1` or `tf.summary.histogram` | Histograms, Distributions |
| 이미지 시각화 | `tf.summary.image` | Images |
| 텍스트 로깅 | `tf.summary.text` | Text |
| 하이퍼파라미터 비교 | `tf.summary.experimental.set_step` + HParams 플러그인 | HParams |

### 권장 워크플로우

```python
# 1. 타임스탬프 기반 로그 디렉토리 (실험 구분)
log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'

# 2. 콜백 설정
tb_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# 3. 학습
model.fit(..., callbacks=[tb_cb])

# 4. 터미널에서 실행
# tensorboard --logdir logs
```